In [9]:
import numpy as np
import os
import glob
import shutil
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
from ipynb.fs.full.chess_functions import *

In [10]:
folders_dict = {
    "r": "blackRook",
    "R": "whiteRook",
    "b": "blackBishop",
    "B": "whiteBishop",
    "n": "blackKnight",
    "N": "whiteKnight",
    "k": "blackKing",
    "K": "whiteKing",
    "q": "blackQueen",
    "Q": "whiteQueen",
    "p": "blackPawn",
    "P": "whitePawn",
    "-": "emptyCell"
}

current_directory = os.getcwd()
chess_pieces_dataset_folder_path = os.path.join(current_directory, "ChessPiecesDatasetV4")

In [11]:
def create_dataset_folders():
    current_directory = os.getcwd()
    directory = "ChessPiecesDatasetV4"
    
    path = os.path.join(current_directory, directory)
    train_path = os.path.join(current_directory, f"{directory}\\train")
    test_path = os.path.join(current_directory, f"{directory}\\test")
    validation_path = os.path.join(current_directory, f"{directory}\\validation")
    
    if os.path.exists(directory):
        print('Deleting directory...')
        shutil.rmtree(path)

    print('Making directory...')
    os.mkdir(path)
    
    # create train and test subfolders
    subfolder_names = ['blackRook', 'whiteRook', 'blackBishop', 'whiteBishop', 'blackKnight', 'whiteKnight', 'blackKing', 'whiteKing', 'blackQueen', 'whiteQueen', 'blackPawn', 'whitePawn', 'emptyCell' ]
    
    for subfolder_name in subfolder_names:
        os.makedirs(os.path.join(train_path, subfolder_name))
        os.makedirs(os.path.join(test_path, subfolder_name))
        os.makedirs(os.path.join(validation_path, subfolder_name))

In [12]:
def add_cell_image_to_folder(cell_image, piece, image_name, main_folder_name):
    current_directory = os.getcwd()
    path = os.path.join(chess_pieces_dataset_folder_path, f"{main_folder_name}\\{folders_dict[piece]}\\{image_name}")
    image = cv2.cvtColor(cell_image, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(path, image)

In [13]:
def extract_cells_from_image(img_path, fen_parser, main_folder_name):
    bgr_img = cv2.imread(img_path)
    image = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    #plt.imshow(image)
    #plt.show()
    
    board_matrix = []
    
    # fen matrix
    image_fen = img_path.split("\\")[1].split(".")[0]
    image_fen_matrix = fen_parser.fenToMatrix(image_fen)  
    
    # width and height of one cell (chess board is 8x8)
    stepX = image.shape[1] // 8
    stepY = image.shape[0] // 8
    
    for y in range(0,8):
        
        row = []
        for x in range(0,8):
            startX = x*stepX
            startY = y*stepY
            endX = (x+1)*stepX
            endY = (y+1)*stepY
            
            row.append((startX, startY, endX, endY))
            
            cell = image[startY: endY, startX: endX]
            cell_image_name = f"{image_fen}--({y},{x}).jpeg"
            add_cell_image_to_folder(cell, image_fen_matrix[y][x], cell_image_name, main_folder_name)
    
    

In [14]:
def generate_dataset():

    '''
        use 100000 small images
        100000 / 64 = 1562.5
        10% = 156.25 test
        10% = 156.25 validation
        80% = 1250 train
    '''
    
    train_size = 1250
    test_size = 156
    validation_size = 156
    valid_start_index = 156
    valid_end_index = 156+156
    
    fen_parser = FenParser("-")

    # return all file paths that match a specific pattern
    train = glob.glob("./Kaggle/dataset/train/*.jpeg")
    test = glob.glob("./Kaggle/dataset/test/*.jpeg")
    validation = glob.glob("./Kaggle/dataset/test/*.jpeg")
    
    train = train[:train_size]
    test = test[:test_size]
    validation = validation[valid_start_index:valid_end_index]
    
    for img_path in train:
        extract_cells_from_image(img_path, fen_parser, "train")
    
    for img_path in test:
        extract_cells_from_image(img_path, fen_parser, "test")
    
    for img_path in validation:
        extract_cells_from_image(img_path, fen_parser, "validation")

In [15]:
create_dataset_folders()
generate_dataset()

Making directory...
